In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from adv_train.scripts.train_adv import AdversarialTraining, Attacker
import argparse
import sys
from pprint import pprint, pformat

In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.cuda as cuda

import numpy as np

from adv_train.launcher import Launcher
from adv_train.model import (
    DatasetType,
    MnistModel,
    CifarModel,
    load_dataset,
    load_classifier,
)
from adv_train.dynamic import Attacker
from adv_train.dataset import AdversarialDataset

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [6]:
sns.set()

In [7]:
sys.argv = ['', '--config','/home/mila/d/david-a.dobre/scratch/2022/Adversarial-Training/configs/experiments/cifar-langevin-resnet18.yml']

In [8]:
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

In [9]:
parser = Attacker.add_arguments(parser)
parser = AdversarialTraining.add_argument(parser)

In [10]:
args = AdversarialTraining.parse_args_with_config(parser)

In [11]:
args.seed=0

In [12]:
pprint(vars(args))

{'attacker_type': <Attacker.LANGEVIN: 'langevin'>,
 'batch_size': 128,
 'config': '/home/mila/d/david-a.dobre/scratch/2022/Adversarial-Training/configs/experiments/cifar-langevin-resnet18.yml',
 'dataset': <DatasetType.CIFAR: 'cifar'>,
 'dest': None,
 'eps_iter': 8.0,
 'eval_adv': <Attacker.PGD_40: 'pgd-40'>,
 'eval_clean_flag': True,
 'eval_name': None,
 'fancy_db_name': True,
 'log_dir': './logs',
 'lr': 0.1,
 'model_dir': None,
 'n_adv': 1,
 'n_epochs': 100,
 'nb_iter': 1,
 'noise_scale': 0.2,
 'noise_type': <NoiseType.NORMAL: 'normal'>,
 'restart': False,
 'save_model': None,
 'seed': 0,
 'sign_flag': True,
 'slurm': '/network/scratch/d/david-a.dobre/2022/Adversarial-Training/configs/slurm/mila-long.yml',
 'train_on_clean': False,
 'type': <CifarModel.RESNET_18: 'res18'>,
 'wandb_entity': 'busycalibrating',
 'wandb_group': 'cifar_langevin_1',
 'wandb_name': None,
 'wandb_project': 'adversarial_training'}


## Prepare trainer

In [13]:
adv_train = AdversarialTraining(args)

In [14]:
adv_train.num_grad = 0
torch.manual_seed(adv_train.seed)
adv_train.device = "cuda" if torch.cuda.is_available() else "cpu"

### Load and prepare datasets, dataloaders, and attackers

In [15]:
dataset = load_dataset(adv_train.dataset, train=True)

Files already downloaded and verified


In [16]:
adv_train._dataset = AdversarialDataset(dataset, n_adv=adv_train.n_adv)
adv_train.dataloader = DataLoader(adv_train._dataset, batch_size=adv_train.batch_size, shuffle=True, num_workers=4)

test_dataset = load_dataset(adv_train.dataset, train=False)
adv_train.test_dataloader = DataLoader(test_dataset, batch_size=adv_train.batch_size, shuffle=False, num_workers=4)

Files already downloaded and verified


### Prepare model and optimzers

In [17]:
adv_train.model = load_classifier(adv_train.dataset, adv_train.type, device=adv_train.device, eval=False)

In [18]:
adv_train.opt = optim.SGD(adv_train.model.parameters(), lr=adv_train.lr)
adv_train.loss_fn = nn.CrossEntropyLoss()
adv_train.attacker = Attacker.load_attacker(adv_train.model, adv_train.args)

In [19]:
adv_train.attacker

# Run Langevin attacker over all training data

In [20]:
train_err, train_loss = adv_train.epoch_adversarial_lan()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:50<00:00,  7.70it/s]


In [21]:
print(train_err, train_loss)

72.974 0.015341595196723939


In [22]:
adv_train._dataset.adv[0]

tensor([[[[-0.5060, -0.6940, -0.5766,  ...,  0.2080,  0.2234,  0.1295],
          [-0.8433, -0.9688, -0.8901,  ..., -0.0665, -0.0979, -0.0744],
          [-0.7727, -0.9058, -0.5844,  ..., -0.1058, -0.0901, -0.1138],
          ...,
          [ 0.6001,  0.5452,  0.5842,  ...,  0.2862, -0.5295, -0.6156],
          [ 0.3805,  0.3256,  0.4276,  ...,  0.4119, -0.2705, -0.3803],
          [ 0.4195,  0.2864,  0.3727,  ...,  0.6629,  0.1531, -0.0665]],

         [[-0.5450, -0.6705, -0.5923,  ...,  0.0040, -0.0509,  0.0038],
          [-0.8744, -0.9688, -0.9685,  ..., -0.2786, -0.3178, -0.3489],
          [-0.8430, -0.9763, -0.7570,  ..., -0.3099, -0.3724, -0.3962],
          ...,
          [ 0.3021,  0.1688,  0.2315,  ...,  0.0744, -0.7881, -0.7646],
          [ 0.0589, -0.0040,  0.0982,  ...,  0.1295, -0.5450, -0.5531],
          [ 0.1607,  0.0430,  0.0825,  ...,  0.4119, -0.1058, -0.3097]],

         [[-0.5371, -0.6158, -0.6940,  ..., -0.1217, -0.1687, -0.1609],
          [-0.8119, -0.9688, -

In [23]:
adv_train._dataset[0]

(tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
          [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
          [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
          ...,
          [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
          [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
          [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],
 
         [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
          [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
          [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
          ...,
          [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
          [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
          [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],
 
         [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
          [-0.8431, -1.0000,

In [45]:
size = len(adv_train._dataset) 

idx = np.arange(size)
idx2 = np.zeros(size)

In [46]:
noise = adv_train._dataset.compute_noise(idx, idx2)

In [62]:
data = noise.flatten() # / noise.absolute().max()

In [48]:
# plt.figure(figsize=(8, 6), dpi=80)
# sns.displot((noise.flatten() / noise.absolute().max()))

In [49]:
# data.numpy()

In [64]:
# len(data)

In [65]:
def value_hist(ds: AdversarialDataset, epoch: int):
    size = len(ds)
    idx = np.arange(size)
    idx2 = np.zeros(size)
    
    noise = ds.compute_noise(idx, idx2)
    data = noise.flatten() # / noise.absolute().max()
    
    # Create just a figure and only one subplot
    fig, ax = plt.subplots(1, 1, dpi=120, figsize=(7,5))
    ax.set_title(f'Perturbation histogram: epoch {epoch}')
    ax.semilogy()
    ax.set_ylim([0, 10**9])
    # ax.plot(x, y)
    sns.histplot(data.numpy(), ax=ax, bins=9)
    return fig

In [66]:
# # Create just a figure and only one subplot
# fig, ax = plt.subplots(1, 1, dpi=120, figsize=(7,5))
# ax.set_title('Blep')
# ax.semilogy()
# ax.set_ylim([0, 10**9])
# # ax.plot(x, y)
# sns.histplot(data.numpy(), ax=ax, bins=9) 

In [ ]:
fig = value_hist(adv_train._dataset, 1)

/tmp/ipykernel_37960/2630979572.py:13: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim([0, 10**9])


In [ ]:
fig.show()

In [61]:
with wandb.init(entity="busycalibrating", project="tests", name="pls_work") as run:
    # tips = sns.load_dataset("tips")
    # fig, ax = plt.subplots(figsize=(11.7, 8.27))
    # sns.scatterplot(data=tips, x="total_bill", y="tip")
    wandb.log({"plot":wandb.Image(fig)})